<a href="https://colab.research.google.com/github/MDYLL/Megafon_test/blob/master/Megafon_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [14]:
from google.colab import files
uploaded = files.upload()

Saving 021_Data.csv to 021_Data (1).csv


In [15]:
ls

'021_Data (1).csv'   021_Data.csv   02_Data.csv   sample_data/


In [0]:
df=pd.read_csv('02_Data.csv',sep=';')

In [19]:
df.head()

,lac,cid,msisdn,imei,event_type,tstamp,long,lat,max_dist,cell_type,start_angle,end_angle
0,5029,40798,158529599791,3.531111e+14,0,1369252800974,37.9305,55.7815,2101,OUTDOOR,220.0,280.0
1,7782,56870,158520145943,1.292900e+13,5,1369252801396,37.4239,55.7426,2,OUTDOOR,45.0,105.0
2,7794,32226,158521798391,3.519940e+14,5,1369252802013,37.7034,55.5803,1125,OUTDOOR,0.0,60.0
3,7758,33528,158537830573,3.586270e+14,0,1369252802331,37.7895,55.7042,1158,OUTDOOR,210.0,270.0
4,5060,17568,158510204039,3.524581e+14,8,1369252803586,37.1900,55.8308,500,OUTDOOR,180.0,240.0


In [20]:
pd.to_datetime(df.tstamp.min(), unit='ms'),pd.to_datetime(df.tstamp.max(), unit='ms')

(Timestamp('2013-05-22 20:00:00.974000'),
 Timestamp('2013-05-30 16:29:37.643000'))

In [0]:
df=df.drop(['lac','cid','imei','event_type','max_dist','cell_type','start_angle','end_angle'],axis=1)
df.tstamp//=1000

In [22]:
df.head()

,msisdn,tstamp,long,lat
0,158529599791,1369252800,37.9305,55.7815
1,158520145943,1369252801,37.4239,55.7426
2,158521798391,1369252802,37.7034,55.5803
3,158537830573,1369252802,37.7895,55.7042
4,158510204039,1369252803,37.1900,55.8308


In [23]:
unique_numbers=df['msisdn'].unique()
len(unique_numbers)

2361

In [0]:
df.tstamp-=df.tstamp.min(axis=0)
quantity_time_intervals=30
min_tstamp=df.tstamp.min(axis=0)
max_tstamp=df.tstamp.max(axis=0)
time_period=(max_tstamp-min_tstamp)//quantity_time_intervals
# time_intervals=[el for el in range(0,max_tstamp,time_period)]
df_norm=pd.DataFrame(columns=df.columns)

In [25]:
df

,msisdn,tstamp,long,lat
0,158529599791,0,37.9305,55.7815
1,158520145943,1,37.4239,55.7426
2,158521798391,2,37.7034,55.5803
3,158537830573,2,37.7895,55.7042
4,158510204039,3,37.1900,55.8308
...,...,...,...,...
174118,158503851673,673967,37.6178,55.7370
174119,158503851673,674287,37.6217,55.7363
174120,158503851673,675316,37.6217,55.7363
174121,158503851673,678091,37.7856,55.7508


In [26]:
min_tstamp,max_tstamp,time_period

(0, 678577, 22619)

In [0]:
import collections
current_tstamp=collections.defaultdict(tuple)
for number in unique_numbers:
  df1=df.loc[df.msisdn==number]
  idx=df1.tstamp.idxmax()
  current_tstamp[number]=(df.at[idx,'long'],df.at[idx,'lat'])


In [59]:
df_norm=pd.DataFrame(columns=df.columns)
for i in range(0,quantity_time_intervals):
  print(i,end=' ')
  for number in unique_numbers:
  # for j in range(1):
    # number=158510204039
    df1=df.loc[(df.msisdn==number)&(df.tstamp<=max_tstamp-i*time_period)&(df.tstamp>max_tstamp-(i+1)*time_period)]
    if df1.size>0:
      idx=df1.tstamp.idxmax()
      df_norm=df_norm.append(df.iloc[[idx]])
      # df_norm.at[idx,'tstamp']=max_tstamp-i*time_period
      df_norm.at[idx,'tstamp']=i
      current_tstamp[number]=(df1.at[idx,'long'],df1.at[idx,'lat'])
      # if number==158510204039:
      #   print(df1.shape)
      #   print(idx)
      #   print(i,current_tstamp[number])
    else:
      # print('**************')
      # print(i)
      # df_norm=df_norm.append({'msisdn':number,'tstamp':max_tstamp-i*time_period,'long':current_tstamp[number][0],'lat':current_tstamp[number][1]},ignore_index=True)
      df_norm=df_norm.append({'msisdn':number,'tstamp':i,'long':current_tstamp[number][0],'lat':current_tstamp[number][1]},ignore_index=True)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

In [79]:
df_norm=df_norm.astype({'msisdn':int,'tstamp':int})
# df_norm.loc[df_norm.msisdn==158510204039]
df_norm

,msisdn,tstamp,long,lat
0,158529599791,0,37.9189,55.7804
1,158520145943,0,37.6034,55.7525
2,158521798391,0,37.7034,55.5803
3,158537830573,0,37.7769,55.6989
4,158510204039,0,37.1256,55.8134
...,...,...,...,...
70825,158599104843,29,37.0184,55.7105
70826,158500569307,29,37.8184,55.8072
70827,158532716103,29,37.7199,55.8921
70828,158503851673,29,37.6542,55.7415


In [0]:
# pair_dist=dict()
# idx=0
# for i in unique_numbers:
#   idx+=1
#   print(idx,end=' ')
#   for j in unique_numbers:
#     if i<j:
#       continue
#     dist=0
#     for k in range(max_tstamp,min_tstamp+time_period,-time_period):
#       long1=df_norm.loc[(df_norm.msisdn==i)&(df_norm.tstamp==k)].long.max()
#       long2=df_norm.loc[(df_norm.msisdn==j)&(df_norm.tstamp==k)].long.max()
#       lat1=df_norm.loc[(df_norm.msisdn==i)&(df_norm.tstamp==k)].lat.max()
#       lat2=df_norm.loc[(df_norm.msisdn==j)&(df_norm.tstamp==k)].lat.max()
#       dist+=((long1-long2)*(long1-long2)+(lat1-lat2)*(lat1-lat2))**0.5
#     pair_dist[(i,j)]=dist

In [0]:
# df_norm1=df_norm.copy()

In [0]:
# df_norm1=df_norm1.rename(columns={'msisdn':'msisdn1','tstamp':'tstamp1','long':'long1','lat':'lat1'})

In [0]:
# a.assign(key=1).merge(b.assign(key=1), on='key').drop('key',axis=1)
# df_pairs=df_norm.assign(key=1).merge(df_norm1.assign(key=1),on='key').drop('key',axis=1)

In [62]:
import torch
(int(len(unique_numbers)*(len(unique_numbers)-1)/2),2*2*quantity_time_intervals)

(2785980, 120)

In [0]:
x=torch.zeros((int(len(unique_numbers)*(len(unique_numbers)-1)/2),2*2*quantity_time_intervals))

In [64]:
x.shape

torch.Size([2785980, 120])

In [98]:
# x[0]=torch.tensor([el for el in range(120)])
idx=0
row=[el for el in range(120)]
x[idx]=torch.tensor(row)
i=unique_numbers[0]
k=7
df_norm.loc[df_norm['msisdn']==i]
# a=df_norm.loc[(df_norm['msisdn']==i)&(df_norm['tstamp']==k)]['lat'].values[0]
# row=[]
# row.append(df_norm.loc[(df_norm['msisdn']==i)&(df_norm['tstamp']==k)]['lat'].values[0])
# row

,msisdn,tstamp,long,lat
0,158529599791,0,37.9189,55.7804
2361,158529599791,1,37.9189,55.7804
4722,158529599791,2,37.9189,55.7804
7083,158529599791,3,37.9189,55.7804
9444,158529599791,4,37.9189,55.7804
11805,158529599791,5,37.9189,55.7804
14166,158529599791,6,37.9189,55.7804
16527,158529599791,28,37.9189,55.7804
18888,158529599791,8,37.9189,55.7804
21249,158529599791,9,37.9189,55.7804


In [95]:
idx=0
for i in unique_numbers:
  for j in unique_numbers:
    if i<=j:
      continue
    row=[]
    for k in range(quantity_time_intervals):
      print(k,end=' ')
      row.append(df_norm.loc[(df_norm['msisdn']==i)&(df_norm['tstamp']==k)]['lat'].values[0])
      row.append(df_norm.loc[(df_norm['msisdn']==i)&(df_norm['tstamp']==k)]['long'].values[0])
    for k in range(quantity_time_intervals):
      row.append(df_norm.loc[(df_norm['msisdn']==j)&(df_norm['tstamp']==k)]['lat'].values[0])
      row.append(df_norm.loc[(df_norm['msisdn']==j)&(df_norm['tstamp']==k)]['long'].values[0])
    x[idx]=torch.tensor(row)
    idx+=1
    print(idx,end=' ')
    
    

0 1 2 3 4 5 6 7 

IndexError: ignored

In [94]:
len(row)

14